In [1]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

from typing import Dict

import pandas as pd
pd.set_option("display.max_columns", 30)

import requests

In [2]:
""""
Steps for the Transform Load Lambda function
1. Get data from S3 (taxi, weather)
2. Weather data transformations -  DONE
3. Taxi data transformations  -  DONE
4. Update dim_payment_type   - DONE
5. Update dim_company  - DONE
6. Update fact_taxi_trips with the ids from the dim_payment_type and dim_company - DONE 
7. Upload dim_weather to S3
8. Upload fact_taxi_trips to S3
9. Upload dim_payment_type and dim_company (current, and previous version)

"""

'"\nSteps for the Transform Load Lambda function\n1. Get data from S3 (taxi, weather)\n2. Weather data transformations -  DONE\n3. Taxi data transformations  -  DONE\n4. Update dim_payment_type   - DONE\n5. Update dim_company  - DONE\n6. Update fact_taxi_trips with the ids from the dim_payment_type and dim_company - DONE \n7. Upload dim_weather to S3\n8. Upload fact_taxi_trips to S3\n9. Upload dim_payment_type and dim_company (current, and previous version)\n\n'

In [3]:
current_datetime = datetime.now() - relativedelta(months=2)
formatted_datetime = current_datetime.strftime("%Y-%m-%d")

url = (
    f"https://data.cityofchicago.org/resource/ajtu-isnz.json?"
    f"$where=trip_start_timestamp >= '{formatted_datetime}T00:00:00' " 
    f"AND trip_start_timestamp <= '{formatted_datetime}T23:59:59' "
    f"&$limit=50000"
)

response = requests.get(url)
data = response.json()

taxi_trips = pd.DataFrame(data)
taxi_trips.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_community_area,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,pickup_census_tract,dropoff_census_tract
0,59a0800cc5f8991af321c720a4b76cbf06786282,70bca78835e8e92b04b46d111508f56c7b847abad90561...,2025-10-13T23:45:00.000,2025-10-14T00:00:00.000,1064,13.37,33,33.5,0,0,0,33.5,Prcard,Flash Cab,41.857183858,-87.620334624,"{'type': 'Point', 'coordinates': [-87.62033462...",NaN,NaN,NaN,NaN,NaN,NaN
1,5aa6b72bc2215836b6119f38c9fe8c7b1e012383,b3d64534f81ff06a34ae1001f3f6e3831011fb0f7a6f40...,2025-10-13T23:45:00.000,2025-10-14T00:15:00.000,1464,13.13,76,34.25,7.95,0,5,47.7,Credit Card,Taxicab Insurance Agency Llc,41.980264315,-87.913624596,"{'type': 'Point', 'coordinates': [-87.91362459...",3,41.96581197,-87.655878786,"{'type': 'Point', 'coordinates': [-87.65587878...",NaN,NaN
2,5b7580464c3b3d585ddd240396b85568876ec972,9a423881d4046f5590943c08ec5e58d59f5ebd91fe26aa...,2025-10-13T23:45:00.000,2025-10-14T00:15:00.000,1440,20.9,76,50,4.5,0,45,99.5,Credit Card,Transit Administrative Center Inc,41.980264315,-87.913624596,"{'type': 'Point', 'coordinates': [-87.91362459...",NaN,NaN,NaN,NaN,NaN,NaN
3,5da637eb009ee06d97ad426bf33c7aff162ade9f,16a529a73f019759aa8f1b563ab8511b26a91d65659188...,2025-10-13T23:45:00.000,2025-10-14T00:15:00.000,1680,17.9,76,44.5,0,0,4,48.5,Cash,Transit Administrative Center Inc,41.97907082,-87.903039661,"{'type': 'Point', 'coordinates': [-87.90303966...",32,41.884987192,-87.620992913,"{'type': 'Point', 'coordinates': [-87.62099291...",17031980000,17031320100
4,6507318fa5786475808ed6de6fd76013c9968c83,6bc3520aa3e5055fdd502ad4db2b419e40a2579182d817...,2025-10-13T23:45:00.000,2025-10-14T00:00:00.000,1251,13.12,76,32.75,7.45,0,4,44.7,Credit Card,Sun Taxi,41.980264315,-87.913624596,"{'type': 'Point', 'coordinates': [-87.91362459...",6,41.944226601,-87.655998182,"{'type': 'Point', 'coordinates': [-87.65599818...",NaN,NaN


#### Taxi data transformations

In [4]:
def taxi_trips_transformations(taxi_trips: pd.DataFrame) -> pd.DataFrame:
    """Perform tranformations on the taxi data.

    1. Drop selected columns.
    2. Drop NULL values accross all columns.
    3. Rename selected columns.
    4. Create "datetime_for_weather" helper column (for dim_weather join).

    :param taxi_trips:  The DataFrame holding the daily taxi trips.
    :raises TypeError:  When taxi_trips parameter is not a valid pandas DataFrame.
    :return:            Transformed taxi trips DataFrame.
    """
    if not isinstance(taxi_trips, pd.DataFrame):
        raise TypeError("taxi_trips is not a valid pandas DataFrame.")

    taxi_trips.drop(["pickup_census_tract", "dropoff_census_tract", 
                 "pickup_centroid_location", "dropoff_centroid_location"], axis=1, inplace=True)
    taxi_trips.dropna(inplace=True)

    taxi_trips.rename(columns={"pickup_community_area": "pickup_community_area_id",
                            "dropoff_community_area": "dropoff_community_area_id"}, inplace=True)

    taxi_trips["trip_start_timestamp"] = pd.to_datetime(taxi_trips["trip_start_timestamp"])
    taxi_trips["datetime_for_weather"] = taxi_trips["trip_start_timestamp"].dt.floor("h")

    return taxi_trips

In [5]:
taxi_trips_transformed = taxi_trips_transformations(taxi_trips)

taxi_trips_transformed.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_community_area_id,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather
1,5aa6b72bc2215836b6119f38c9fe8c7b1e012383,b3d64534f81ff06a34ae1001f3f6e3831011fb0f7a6f40...,2025-10-13 23:45:00,2025-10-14T00:15:00.000,1464,13.13,76,34.25,7.95,0,5,47.7,Credit Card,Taxicab Insurance Agency Llc,41.980264315,-87.913624596,3,41.96581197,-87.655878786,2025-10-13 23:00:00
3,5da637eb009ee06d97ad426bf33c7aff162ade9f,16a529a73f019759aa8f1b563ab8511b26a91d65659188...,2025-10-13 23:45:00,2025-10-14T00:15:00.000,1680,17.9,76,44.5,0,0,4,48.5,Cash,Transit Administrative Center Inc,41.97907082,-87.903039661,32,41.884987192,-87.620992913,2025-10-13 23:00:00
4,6507318fa5786475808ed6de6fd76013c9968c83,6bc3520aa3e5055fdd502ad4db2b419e40a2579182d817...,2025-10-13 23:45:00,2025-10-14T00:00:00.000,1251,13.12,76,32.75,7.45,0,4,44.7,Credit Card,Sun Taxi,41.980264315,-87.913624596,6,41.944226601,-87.655998182,2025-10-13 23:00:00
5,66e8d62f25be92f840451673799f2dbca429ff98,84957c8960b674346784746bbc1d48cafff4976b162323...,2025-10-13 23:45:00,2025-10-14T00:00:00.000,673,4.64,6,14.25,0,0,0,14.25,Prcard,Flash Cab,41.944226601,-87.655998182,8,41.899602111,-87.633308037,2025-10-13 23:00:00
6,66f71db665b721fccad5057f9c312feea271a410,b3d64534f81ff06a34ae1001f3f6e3831011fb0f7a6f40...,2025-10-13 23:45:00,2025-10-13T23:45:00.000,276,0.76,76,5.5,0,0,0,5.5,Cash,Taxicab Insurance Agency Llc,41.97907082,-87.903039661,76,41.97907082,-87.903039661,2025-10-13 23:00:00


In [6]:
taxi_trips_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18261 entries, 1 to 20367
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   trip_id                     18261 non-null  object        
 1   taxi_id                     18261 non-null  object        
 2   trip_start_timestamp        18261 non-null  datetime64[ns]
 3   trip_end_timestamp          18261 non-null  object        
 4   trip_seconds                18261 non-null  object        
 5   trip_miles                  18261 non-null  object        
 6   pickup_community_area_id    18261 non-null  object        
 7   fare                        18261 non-null  object        
 8   tips                        18261 non-null  object        
 9   tolls                       18261 non-null  object        
 10  extras                      18261 non-null  object        
 11  trip_total                  18261 non-null  object        


#### Dim Compyna and Dim Payment Type update

In [7]:
def update_dim_table(taxi_trips: pd.DataFrame, dim_df: pd.DataFrame, value_col: str) -> pd.DataFrame:
    """Extend the dimension DataFrame with new values if there are any.

    :param taxi_trips:  DataFrame with the daily taxi trips.
    :param dim_df:      DataFrame with the dimension data (company, payment_type).
    :param value_col:   Name of the column in dimension DataFrame containing the values.
    :return:            The updated dimension data, if new values are in the taxi data, they will be loaded to it.
    """
    id_col = f"{value_col}_id"

    todays_dim_data = pd.DataFrame(taxi_trips[value_col].unique(), columns=[value_col])
    new_dim_data = todays_dim_data[~todays_dim_data[value_col].isin(dim_df[value_col])]

    if not new_dim_data.empty:
        max_id = dim_df[id_col].max()
        new_dim_data[id_col] = range(max_id + 1, max_id + 1 + len(new_dim_data))
        dim_df = pd.concat([dim_df, new_dim_data], ignore_index=True)

    return dim_df

In [8]:
dim_payment_type = taxi_trips["payment_type"].drop_duplicates().reset_index(drop=True)
dim_payment_type = pd.DataFrame(
    {
        "payment_type_id": range(1, len(dim_payment_type) + 1),
        "payment_type": dim_payment_type
    }
)

dim_company = taxi_trips["company"].drop_duplicates().reset_index(drop=True)
dim_company = pd.DataFrame(
    {
        "company_id": range(1, len(dim_company) + 1),
        "company": dim_company
    }
)

dummy_company_data = [
    {"company": "Metro Jet Taxi A."},
    {"company": "X"},
    {"company": "Y"},
    {"company": "X"},
]
dummy_company_data_df = pd.DataFrame(dummy_company_data)

dummy_payment_type_data = [
    {"payment_type": "Credit Card"},
    {"payment_type": "X"},
    {"payment_type": "Z"},
    {"payment_type": "Z"},
]
dummy_payment_type_data_df = pd.DataFrame(dummy_payment_type_data)

In [10]:
dim_payment_type_updated = update_dim_table(dummy_payment_type_data_df, dim_payment_type, "payment_type")
dim_company_updated = update_dim_table(dummy_company_data_df, dim_company, "company")

C:\Users\tothj\AppData\Local\Temp\ipykernel_17364\1748550728.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dim_data[id_col] = range(max_id + 1, max_id + 1 + len(new_dim_data))
C:\Users\tothj\AppData\Local\Temp\ipykernel_17364\1748550728.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dim_data[id_col] = range(max_id + 1, max_id + 1 + len(new_dim_data))


In [11]:
dim_payment_type_updated


,payment_type_id,payment_type
0,1,Credit Card
1,2,Cash
2,3,Prcard
3,4,Mobile
4,5,Unknown
5,6,No Charge
6,7,Dispute
7,8,X
8,9,Z


In [12]:
dim_company_updated

,company_id,company
0,1,Taxicab Insurance Agency Llc
1,2,Transit Administrative Center Inc
2,3,Sun Taxi
3,4,Flash Cab
4,5,5 Star Taxi
5,6,City Service
6,7,Chicago City Taxi Association
7,8,312 Medallion Management Corp
8,9,Chicago Independents
9,10,Medallion Leasin


#### Update fact_taxi_trips with company and payment_type ids

In [13]:
def update_fact_taxi_trips_with_dimension_data(taxi_trips: pd.DataFrame, dim_payment_type: pd.DataFrame, dim_company: pd.DataFrame) -> pd.DataFrame:
    """Update the fact_taxi_trips DataFrame with the dim_company and dim_payment_type ids, and delete the string columns.

    :param taxi_trips:          The DataFrame with the daily taxi trips.
    :param dim_payment_type:    The payment type dimension table.
    :param dim_company:         The company dimension table.
    :return:                    The taxi trips data, with only payment_type_id and company_id, without company or
                                payment_type values.
    """

    fact_taxi_trips = taxi_trips.merge(dim_payment_type, on="payment_type")
    fact_taxi_trips = fact_taxi_trips.merge(dim_company, on="company")
    fact_taxi_trips.drop(["payment_type", "company"], axis=1, inplace=True)

    return fact_taxi_trips

In [14]:
taxi_trips_transformed_with_dim_ids = update_fact_taxi_trips_with_dimension_data(taxi_trips_transformed, dim_payment_type, dim_company)

taxi_trips_transformed_with_dim_ids.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_community_area_id,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,payment_type_id,company_id
0,5aa6b72bc2215836b6119f38c9fe8c7b1e012383,b3d64534f81ff06a34ae1001f3f6e3831011fb0f7a6f40...,2025-10-13 23:45:00,2025-10-14T00:15:00.000,1464,13.13,76,34.25,7.95,0,5,47.7,41.980264315,-87.913624596,3,41.96581197,-87.655878786,2025-10-13 23:00:00,1,1
1,5da637eb009ee06d97ad426bf33c7aff162ade9f,16a529a73f019759aa8f1b563ab8511b26a91d65659188...,2025-10-13 23:45:00,2025-10-14T00:15:00.000,1680,17.9,76,44.5,0,0,4,48.5,41.97907082,-87.903039661,32,41.884987192,-87.620992913,2025-10-13 23:00:00,2,2
2,6507318fa5786475808ed6de6fd76013c9968c83,6bc3520aa3e5055fdd502ad4db2b419e40a2579182d817...,2025-10-13 23:45:00,2025-10-14T00:00:00.000,1251,13.12,76,32.75,7.45,0,4,44.7,41.980264315,-87.913624596,6,41.944226601,-87.655998182,2025-10-13 23:00:00,1,3
3,66e8d62f25be92f840451673799f2dbca429ff98,84957c8960b674346784746bbc1d48cafff4976b162323...,2025-10-13 23:45:00,2025-10-14T00:00:00.000,673,4.64,6,14.25,0,0,0,14.25,41.944226601,-87.655998182,8,41.899602111,-87.633308037,2025-10-13 23:00:00,3,4
4,66f71db665b721fccad5057f9c312feea271a410,b3d64534f81ff06a34ae1001f3f6e3831011fb0f7a6f40...,2025-10-13 23:45:00,2025-10-13T23:45:00.000,276,0.76,76,5.5,0,0,0,5.5,41.97907082,-87.903039661,76,41.97907082,-87.903039661,2025-10-13 23:00:00,2,1


#### Weather transformations


In [15]:
def transform_weather(data: Dict) -> pd.DataFrame:
    """Select and transform weather data.

    :param weather_data:    The daily weather data from the Open Meteo API.
    :return:                Transformed weather pandas DataFrame.
    """

    weather_data_dict = {
        "datetime": data["hourly"]["time"],
        "temperature": data["hourly"]["temperature_2m"],
        "wind_speed": data["hourly"]["wind_speed_10m"],
        "rain": data["hourly"]["rain"],
        "precipitation": data["hourly"]["precipitation"]
    }

    weather_df = pd.DataFrame(weather_data_dict)
    weather_df["datetime"] = pd.to_datetime(weather_df["datetime"])

    return weather_df

In [16]:
current_datetime = datetime.now() - relativedelta(months=2)
formatted_datetime = current_datetime.strftime("%Y-%m-%d")

url = "https://archive-api.open-meteo.com/v1/era5"

params = {
    "latitude": 41.85,
    "longitude": -87.65,
    "start_date": formatted_datetime,
    "end_date": formatted_datetime,
    "hourly": "temperature_2m,wind_speed_10m,rain,precipitation"
}

response = requests.get(url, params=params)
weather_raw_data = response.json()

weather_raw_data

{'latitude': 41.862915,
 'longitude': -87.64877,
 'generationtime_ms': 0.48792362213134766,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 179.0,
 'hourly_units': {'time': 'iso8601',
  'temperature_2m': '°C',
  'wind_speed_10m': 'km/h',
  'rain': 'mm',
  'precipitation': 'mm'},
 'hourly': {'time': ['2025-10-13T00:00',
   '2025-10-13T01:00',
   '2025-10-13T02:00',
   '2025-10-13T03:00',
   '2025-10-13T04:00',
   '2025-10-13T05:00',
   '2025-10-13T06:00',
   '2025-10-13T07:00',
   '2025-10-13T08:00',
   '2025-10-13T09:00',
   '2025-10-13T10:00',
   '2025-10-13T11:00',
   '2025-10-13T12:00',
   '2025-10-13T13:00',
   '2025-10-13T14:00',
   '2025-10-13T15:00',
   '2025-10-13T16:00',
   '2025-10-13T17:00',
   '2025-10-13T18:00',
   '2025-10-13T19:00',
   '2025-10-13T20:00',
   '2025-10-13T21:00',
   '2025-10-13T22:00',
   '2025-10-13T23:00'],
  'temperature_2m': [16.8,
   17.0,
   16.8,
   16.3,
   15.8,
   15.1,
   14.3,
   13.8,
   13.4,
   13

In [17]:
weather_df = transform_weather(weather_raw_data)

weather_df

,datetime,temperature,wind_speed,rain,precipitation
0,2025-10-13 00:00:00,16.8,11.4,0.0,0.0
1,2025-10-13 01:00:00,17.0,12.7,0.0,0.0
2,2025-10-13 02:00:00,16.8,13.7,0.0,0.0
3,2025-10-13 03:00:00,16.3,13.8,0.0,0.0
4,2025-10-13 04:00:00,15.8,11.6,0.0,0.0
5,2025-10-13 05:00:00,15.1,8.8,0.0,0.0
6,2025-10-13 06:00:00,14.3,9.0,0.0,0.0
7,2025-10-13 07:00:00,13.8,8.5,0.0,0.0
8,2025-10-13 08:00:00,13.4,9.5,0.0,0.0
9,2025-10-13 09:00:00,13.1,8.4,0.0,0.0


In [18]:
weather_df

,datetime,temperature,wind_speed,rain,precipitation
0,2025-10-13 00:00:00,16.8,11.4,0.0,0.0
1,2025-10-13 01:00:00,17.0,12.7,0.0,0.0
2,2025-10-13 02:00:00,16.8,13.7,0.0,0.0
3,2025-10-13 03:00:00,16.3,13.8,0.0,0.0
4,2025-10-13 04:00:00,15.8,11.6,0.0,0.0
5,2025-10-13 05:00:00,15.1,8.8,0.0,0.0
6,2025-10-13 06:00:00,14.3,9.0,0.0,0.0
7,2025-10-13 07:00:00,13.8,8.5,0.0,0.0
8,2025-10-13 08:00:00,13.4,9.5,0.0,0.0
9,2025-10-13 09:00:00,13.1,8.4,0.0,0.0
